 Theunderlying SC was constructed by (i) drawing 400 random points in the unit square;(ii) generating a triangular lattice via Delauney triangulation; (iii) eliminating edgesinside two predefined regions; and (iv) defining all triangles to be faces.

In [ ]:
import pickle
import random

import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy as sc
from scipy.sparse import linalg
from scipy.spatial import Delaunay
from scipy.spatial.distance import cdist

from simplicial_kuramoto import SimplicialComplex, plotting
from simplicial_kuramoto.graph_generator import modular_graph
from simplicial_kuramoto.integrators import *

In [ ]:
def get_delauney_holes_multi(n_points, centre_holes, radius, points=[]):

    if not len(points):
        x = np.random.rand(n_points)
        y = np.random.rand(n_points)
        points = np.vstack([x, y]).T

    tri = Delaunay(points)

    edge_list = []

    idx_inside = np.empty([0], dtype=int)
    for i in range(centre_holes.shape[0]):
        idx_inside = np.hstack(
            [idx_inside, encloses([centre_holes[i]], points, radius)[1]]
        )

    for t in tri.simplices:

        if t[0] not in idx_inside and t[1] not in idx_inside:
            edge_list.append([t[0], t[1]])

        if t[1] not in idx_inside and t[2] not in idx_inside:
            edge_list.append([t[1], t[2]])

        if t[0] not in idx_inside and t[2] not in idx_inside:
            edge_list.append([t[0], t[2]])

    graph = nx.Graph()
    # graph.add_nodes_from(np.arange(len(points)))
    graph.add_edges_from(edge_list)

    Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
    g = graph.subgraph(Gcc[0])

    return g, points


def get_delauney(n_points):

    x = np.random.rand(n_points)
    y = np.random.rand(n_points)
    points = np.vstack([x, y]).T

    tri = Delaunay(points)
    edge_list = []

    for t in tri.simplices:
        edge_list.append([t[0], t[1]])
        edge_list.append([t[1], t[2]])
        edge_list.append([t[0], t[2]])

    graph = nx.Graph()
    # graph.add_nodes_from(np.arange(len(points)))
    graph.add_edges_from(edge_list)

    Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
    g = graph.subgraph(Gcc[0])

    return g, points


def encloses(centre, points, radius):
    inside_hole = cdist(centre, points, "euclidean") <= radius
    idx_inside = np.where(inside_hole)

    return idx_inside

In [ ]:
# Plotting the nullspace of L1


def Null_Space_Plot(graph):
    complex_delaunay = SimplicialComplex(graph=graph, no_faces=False)

    L1 = complex_delaunay.L1

    KerL1 = sc.linalg.null_space(L1.todense())

    for i in range(KerL1.shape[1]):
        plt.figure()
        nx.draw_networkx_nodes(graph, pos=points, node_size=5)
        nx.draw_networkx_edges(
            graph,
            pos=points,
            edge_color=KerL1[:, i],
            edge_cmap=plt.get_cmap("twilight_shifted"),
            width=5,
            edge_vmin=np.min(KerL1[:, i]),
            edge_vmax=np.max(KerL1[:, i]),
        )
        plt.title("Null space of L1, vector " + str(i))
        plt.show()


def extract_spectral_gap(graph):
    # taking the smallest non zero eigenvalue

    if type(graph) == nx.Graph:
        complex_delaunay = SimplicialComplex(graph=graph, no_faces=False)
    else:
        complex_delaunay = graph

    L1 = complex_delaunay.L1

    u, s, vh = sc.linalg.svd(L1.todense(), full_matrices=True)
    M, N = u.shape[0], vh.shape[1]

    rcond = np.finfo(s.dtype).eps * max(M, N)
    tol = np.amax(s) * rcond

    num = np.sum(s > tol, dtype=int)
    Q = vh[num:, :].T.conj()

    spectral_gap = s[num - 1]

    return spectral_gap

In [ ]:
centre_hole_1 = np.array([[0.25, 0.25]])
centre_hole_2 = np.array([[0.75, 0.75]])
centre_hole_3 = np.array([[0.25, 0.75]])

centre_holes = np.concatenate((centre_hole_1, centre_hole_2, centre_hole_3), axis=0)

radius = 0.1

graph, points = get_delauney_holes_multi(400, centre_holes, radius)
pos = dict(enumerate(points))
nx.draw(graph, pos, node_size=30)

print("Spectral gap: {}".format(extract_spectral_gap(graph)))

In [ ]:
centre_hole_1 = np.array([[0.25, 0.25]])
centre_hole_2 = np.array([[0.75, 0.75]])

centre_holes = np.concatenate((centre_hole_1, centre_hole_2), axis=0)

radius = 0.1

graph, points = get_delauney_holes_multi(400, centre_holes, radius)
pos = dict(enumerate(points))
nx.draw(graph, pos, node_size=30)

print("Spectral gap: {}".format(extract_spectral_gap(graph)))

# Single hole size effect

In [ ]:
centre_hole = np.array([[0.5, 0.5]])

n_points = 400
x = np.random.rand(n_points)
y = np.random.rand(n_points)
points = np.vstack([x, y]).T

radii = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45]

print(radii)
s_gaps = []
for radius in radii:
    graph, points = get_delauney_holes_multi(n_points, centre_hole, radius, points)

    s_gaps.append(extract_spectral_gap(graph))


plt.plot(radii, s_gaps)

In [ ]:
centre_hole = np.array([[0.5, 0.5]])

n_points = 400
x = np.random.rand(n_points)
y = np.random.rand(n_points)
points = np.vstack([x, y]).T

radii = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45]

s_gaps = []
order_parameters = []
for radius in radii:
    graph, points = get_delauney_holes_multi(n_points, centre_hole, radius, points)

    s_gaps.append(extract_spectral_gap(graph))

    np.random.seed(0)
    initial_phase = np.random.uniform(np.pi / 2, 3 * np.pi / 2, len(graph.edges))

    t_max = 500
    n_t = 10

    complex_delaunay = SimplicialComplex(graph=graph, no_faces=False)

    edge_result = integrate_edge_kuramoto(complex_delaunay, initial_phase, t_max, n_t)
    # plt.savefig("phases_no_faces.png")

    op = plotting.plot_order_parameter(edge_result.y, return_op=True, plot=False)
    order_parameters.append(op)

In [ ]:
orders = [o[-1] for o in order_parameters]
plt.plot(radii, s_gaps)

plt.plot(radii, orders)

plt.plot(radii, orders)

In [ ]:
complex_delaunay.W1

#  Removing faces but not edges

In [ ]:
graph, points = get_delauney(400)
pos = dict(enumerate(points))

print("Spectral gap: {}".format(extract_spectral_gap(graph)))


all_cliques = nx.enumerate_all_cliques(graph)
faces = [clique for clique in all_cliques if len(clique) == 3]

face_coords = np.array(
    [
        np.mean(np.vstack([points[node] for node in face]), axis=0).tolist()
        for face in faces
    ]
)
face_distance_centre = np.sqrt(((face_coords - 0.5) ** 2).sum(axis=1))
order_distances = np.argsort(face_distance_centre)[::-1]  # reverse order

In [ ]:
s_gaps = []

for i in range(1, len(order_distances)):

    faces_idx = order_distances[:-i]
    faces_to_add = [faces[j] for j in faces_idx]

    if faces_to_add:
        complex_delaunay = SimplicialComplex(
            graph=graph, no_faces=False, faces=faces_to_add
        )
    else:
        complex_delaunay = SimplicialComplex(graph=graph, no_faces=True)

    s_gaps.append(extract_spectral_gap(complex_delaunay))

In [ ]:
plt.plot(s_gaps[::-1])
plt.xlabel("No. central faces removed")
plt.ylabel("Spectral Gap")

# Increasing weight from no-hole to fully filled hole

In [ ]:
def get_weighted_delauney(tri, centre_holes, radius, weight_hole):

    # find idx of nodes inside hole
    idx_inside = np.empty([0], dtype=int)
    for i in range(centre_holes.shape[0]):
        idx_inside = np.hstack(
            [idx_inside, encloses([centre_holes[i]], points, radius)[1]]
        )

    # make real edge list
    edge_list = []
    for t in tri.simplices:
        edge_list.append([t[0], t[1]])
        edge_list.append([t[1], t[2]])
        edge_list.append([t[0], t[2]])

    # make edge list of edges to change weight
    edges_to_weight = []
    for t in tri.simplices:
        if t[0] in idx_inside or t[1] in idx_inside:
            edges_to_weight.append([t[0], t[1]])
        if t[1] in idx_inside or t[2] in idx_inside:
            edges_to_weight.append([t[1], t[2]])
        if t[0] in idx_inside or t[2] in idx_inside:
            edges_to_weight.append([t[0], t[2]])

    # create networkx graph
    graph = nx.Graph()
    graph.add_edges_from(edge_list)
    Gcc = sorted(nx.connected_components(graph), key=len, reverse=True)
    g = graph.subgraph(Gcc[0])

    # add weights
    for u, v in edge_list:
        g[u][v]["weight"] = 1

    for u, v in edges_to_weight:
        g[u][v]["weight"] = weight_hole

    # create simplicial complex fully connected
    complex_delaunay = SimplicialComplex(graph=g, no_faces=False)

    # plot with edge weights
    plt.figure()
    pos = dict(enumerate(points))
    edges, weights = zip(*nx.get_edge_attributes(g, "weight").items())
    nx.draw_networkx_nodes(graph, pos=points, node_size=5)
    nx.draw_networkx_edges(
        graph,
        pos=points,
        edge_color=weights,
        edge_cmap=plt.get_cmap("YlGn"),
        width=2,
        edge_vmin=0,
        edge_vmax=1,
    )

    # define face weights as product of edge weights
    face_weights_matrix = np.ones(complex_delaunay.n_faces)
    for i, face in enumerate(complex_delaunay.faces):

        e1 = complex_delaunay.graph.edges[face[0], face[1]]["weight"]
        e2 = complex_delaunay.graph.edges[face[0], face[2]]["weight"]
        e3 = complex_delaunay.graph.edges[face[1], face[2]]["weight"]

        face_w = e1 * e2 * e3

        face_weights_matrix[i] = face_w

    complex_delaunay.face_weights_matrix = sc.sparse.spdiags(
        face_weights_matrix, 0, complex_delaunay.n_faces, complex_delaunay.n_faces
    )

    return complex_delaunay

In [ ]:
# parameters of delauney
centre_holes = np.array([[0.5, 0.5]])
n_points = 400
radius = 0.2
hole_weight = 0

x = np.random.rand(n_points)
y = np.random.rand(n_points)
points = np.vstack([x, y]).T
tri = Delaunay(points)

complex_delaunay = get_weighted_delauney(tri, centre_holes, radius, hole_weight)

# this function removes edges and faces with zero weight - otherwise we get infinities
complex_delaunay.remove_zero_weight_edges_faces()


print("Spectral gap: {}".format(extract_spectral_gap(complex_delaunay)))

edges, weights = zip(*nx.get_edge_attributes(complex_delaunay.graph, "weight").items())
nx.draw_networkx_nodes(complex_delaunay.graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    complex_delaunay.graph,
    pos=points,
    edge_color=weights,
    edge_cmap=plt.get_cmap("YlGn"),
    width=2,
    edge_vmin=0,
    edge_vmax=1,
)

In [ ]:
# parameters of delauney
centre_holes = np.array([[0.5, 0.5]])
n_points = 400
radius = 0.2
hole_weight = 0.01

x = np.random.rand(n_points)
y = np.random.rand(n_points)
points = np.vstack([x, y]).T
tri = Delaunay(points)

complex_delaunay = get_weighted_delauney(tri, centre_holes, radius, hole_weight)

# this function removes edges and faces with zero weight - otherwise we get infinities
complex_delaunay.remove_zero_weight_edges_faces()


print("Spectral gap: {}".format(extract_spectral_gap(complex_delaunay)))

edges, weights = zip(*nx.get_edge_attributes(complex_delaunay.graph, "weight").items())
nx.draw_networkx_nodes(complex_delaunay.graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    complex_delaunay.graph,
    pos=points,
    edge_color=weights,
    edge_cmap=plt.get_cmap("YlGn"),
    width=2,
    edge_vmin=0,
    edge_vmax=1,
)

In [ ]:
g = complex_delaunay.graph
nx.draw(g, pos=points)
len(g.edges)

In [ ]:
L0 = complex_delaunay.L0.toarray()
np.fill_diagonal(L0, 0)
A = abs(L0)
g = nx.Graph(A)
Gcc = sorted(nx.connected_components(g), key=len, reverse=True)
g = g.subgraph(Gcc[0])
nx.draw(g, pos=nx.spring_layout(g))
len(g.edges)

In [ ]:
# parameters of delauney
centre_holes = np.array([[0.5, 0.5]])
n_points = 400
radius = 0.2
hole_weights = [0, 0.001, 0.05, 0.25, 0.5, 1]  # np.logspace(-3,0,20)

x = np.random.rand(n_points)
y = np.random.rand(n_points)
points = np.vstack([x, y]).T
tri = Delaunay(points)


for hole_weight in hole_weights:

    complex_delaunay = get_weighted_delauney(tri, centre_holes, radius, hole_weight)
    complex_delaunay.remove_zero_weight_edges_faces()
    print("Spectral gap: {}".format(extract_spectral_gap(complex_delaunay)))

In [ ]:
# looping over different edge weights for hole

# parameters of delauney
centre_holes = np.array([[0.5, 0.5]])
n_points = 100
radius = 0.2
hole_weights = [0.001, 0.1, 0.5, 1]  # np.logspace(-3,0,20)

x = np.random.rand(n_points)
y = np.random.rand(n_points)
points = np.vstack([x, y]).T
tri = Delaunay(points)


edge_results = []
for i, hole_weight in enumerate(hole_weights):

    complex_delaunay = get_weighted_delauney(tri, centre_holes, radius, hole_weight)
    complex_delaunay.remove_zero_weight_edges_faces()
    print("Spectral gap: {}".format(extract_spectral_gap(complex_delaunay)))

    np.random.seed(0)
    initial_phase = np.random.uniform(
        np.pi / 2, 3 * np.pi / 2, complex_delaunay.W1.shape[0]
    )

    plt.figure()
    nx.draw_networkx_nodes(complex_delaunay.graph, pos=points, node_size=5)
    nx.draw_networkx_edges(
        complex_delaunay.graph,
        pos=points,
        edge_color=initial_phase,
        edge_cmap=plt.get_cmap("twilight_shifted"),
        width=5,
        edge_vmin=0,
        edge_vmax=np.pi * 2,
    )

    t_max = 500
    n_t = 10

    edge_result = integrate_edge_kuramoto(complex_delaunay, initial_phase, t_max, n_t)

    edge_results.append(edge_result)

    plt.figure()
    nx.draw_networkx_nodes(complex_delaunay.graph, pos=points, node_size=5)
    nx.draw_networkx_edges(
        complex_delaunay.graph,
        pos=points,
        edge_color=edge_results[i].y[:, -1],
        edge_cmap=plt.get_cmap("twilight_shifted"),
        width=5,
        edge_vmin=0,
        edge_vmax=np.pi * 2,
    )

In [ ]:
# setting edge weight to zero for hole and taking the phase

complex_delaunay = get_weighted_delauney(tri, centre_holes, radius, 0)
idx_edges_removed, idx_faces_removed = complex_delaunay.remove_zero_weight_edges_faces(
    return_idx=True
)
complex_delaunay = SimplicialComplex(graph=complex_delaunay.graph, no_faces=False)


phase_hole = initial_phase[np.where(idx_edges_removed)]

edge_result = integrate_edge_kuramoto(complex_delaunay, phase_hole, t_max, n_t)

edge_results = [edge_result] + edge_results

In [ ]:
plt.figure()
nx.draw_networkx_nodes(complex_delaunay.graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    complex_delaunay.graph,
    pos=points,
    edge_color=phase_hole,
    edge_cmap=plt.get_cmap("twilight_shifted"),
    width=5,
    edge_vmin=0,
    edge_vmax=np.pi * 2,
)

In [ ]:
plt.figure()
nx.draw_networkx_nodes(complex_delaunay.graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    complex_delaunay.graph,
    pos=points,
    edge_color=edge_result.y[:, -1],
    edge_cmap=plt.get_cmap("twilight_shifted"),
    width=5,
    edge_vmin=0,
    edge_vmax=np.pi * 2,
)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sim = np.zeros([len(edge_results), len(edge_results)])

for i, er_1 in enumerate(edge_results):
    for j, er_2 in enumerate(edge_results):

        if er_1.y.shape[0] == idx_edges_removed.shape[0]:
            y1 = er_1.y[np.where(idx_edges_removed), -1].reshape(-1, 1).T
        else:
            y1 = er_1.y[:, -1].reshape(-1, 1).T

        if er_2.y.shape[0] == idx_edges_removed.shape[0]:
            y2 = er_2.y[np.where(idx_edges_removed), -1].reshape(-1, 1).T
        else:
            y2 = er_2.y[:, -1].reshape(-1, 1).T

        sim[i, j] = cosine_similarity(y1, y2)

In [ ]:
plt.imshow(sim)
plt.colorbar()

# Comparing the 2nd eigenvector

Compare the 2nd eigenvector for a hole with the 2nd eigenvector for a low edge weight 'hole'.



In [ ]:
# parameters of delauney
centre_holes = np.array([[0.5, 0.5]])
n_points = 100
radius = 0.2
hole_weights = [0.001, 0.1, 0.5, 1]  # np.logspace(-3,0,20)

x = np.random.rand(n_points)
y = np.random.rand(n_points)
points = np.vstack([x, y]).T
tri = Delaunay(points)

In [ ]:
complex_delaunay = get_weighted_delauney(tri, centre_holes, radius, 0)
idx_edges_removed, idx_faces_removed = complex_delaunay.remove_zero_weight_edges_faces(
    return_idx=True
)
complex_delaunay = SimplicialComplex(graph=complex_delaunay.graph, no_faces=False)

eigenvals, eigenvecs = linalg.eigsh(L1, 1, which="SM", return_eigenvectors=True)


plt.figure()
nx.draw_networkx_nodes(complex_delaunay.graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    complex_delaunay.graph,
    pos=points,
    edge_color=eigenvecs.reshape(-1),
    edge_cmap=plt.get_cmap("twilight_shifted"),
    width=5,
)

In [ ]:
complex_delaunay = get_weighted_delauney(tri, centre_holes, radius, 1)
idx_edges_removed, idx_faces_removed = complex_delaunay.remove_zero_weight_edges_faces(
    return_idx=True
)
complex_delaunay = SimplicialComplex(graph=complex_delaunay.graph, no_faces=False)

L1 = complex_delaunay.L1
eigenvals, eigenvecs = linalg.eigsh(L1, 1, which="SM", return_eigenvectors=True)


plt.figure()
nx.draw_networkx_nodes(complex_delaunay.graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    complex_delaunay.graph,
    pos=points,
    edge_color=eigenvecs.reshape(-1),
    edge_cmap=plt.get_cmap("twilight_shifted"),
    width=5,
)

In [ ]:
complex_delaunay = get_weighted_delauney(tri, centre_holes, radius, 0.001)
idx_edges_removed, idx_faces_removed = complex_delaunay.remove_zero_weight_edges_faces(
    return_idx=True
)
complex_delaunay = SimplicialComplex(graph=complex_delaunay.graph, no_faces=False)

L1 = complex_delaunay.L1
eigenvals, eigenvecs = linalg.eigsh(L1, 1, which="SM", return_eigenvectors=True)


plt.figure()
nx.draw_networkx_nodes(complex_delaunay.graph, pos=points, node_size=5)
nx.draw_networkx_edges(
    complex_delaunay.graph,
    pos=points,
    edge_color=eigenvecs.reshape(-1),
    edge_cmap=plt.get_cmap("twilight_shifted"),
    width=5,
)